In [ ]:
%%capture
%run 02_FeatEng.ipynb

In [ ]:
from fbprophet import Prophet
from  sklearn.metrics import mean_squared_error  as skl_mse

In [ ]:
import mle.tsa as mle_tsa

In [ ]:
OUTPATH = 'data/out/'
OUTFILE = 'cons_gna95_fbprophet.xlsx'

# Split Data

In [ ]:
endog_GNA95_train = data_train.loc['2010':, ['cons_GNA95']].copy()
endog_GNA95_train['y'] = np.log(endog_GNA95_train['cons_GNA95'])
endog_GNA95_train['ds'] = endog_GNA95_train.index.strftime("%Y-%m-%d")

In [ ]:
endog_GNA95_test = data_test[['cons_GNA95']].copy()
endog_GNA95_test['y'] = np.log(endog_GNA95_test['cons_GNA95'])
endog_GNA95_test['ds'] = endog_GNA95_test.index.strftime("%Y-%m-%d")

# Model Building
* In prophet multistep forecasting is done by default

In [ ]:
mod_prophet = Prophet()
res_prophet = mod_prophet.fit(df=endog_GNA95_train[['ds', 'y']])

In [ ]:
future = mod_prophet.make_future_dataframe(periods=12, freq='M')
future.tail()

In [ ]:
forecast = res_prophet.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = res_prophet.plot(forecast)
plt.title('GNA95 forecast')
plt.ylabel(r'ln(cons GNA95)')
plt.show()

In [ ]:
fig2 = res_prophet.plot_components(forecast)
plt.suptitle('GNA95 model components: ln(cons GNA95)', y=1.02)
plt.show()

## Prediction

In [ ]:
rename_fbprophet = {'yhat_lower': 'lower p_cons_GNA95', 'yhat_upper': 'upper p_cons_GNA95', 'yhat': 'p_cons_GNA95'}

p_gna95_train = np.exp(forecast[['yhat_lower', 'yhat_upper', 'yhat']]).rename(columns=rename_fbprophet)

p_gna95_train.index = forecast['ds'].dt.date
p_gna95_train.index.name = 'Date' 

p_gna95_train['cons_GNA95'] = endog_GNA95_train['cons_GNA95']

p_gna95_train['model'] = 'fbprophet'
p_gna95_train['split'] = 'train'

# Forecasting

## Multistep Forecasting

In [ ]:
mstep_forecast_test = forecast[-12:]
mstep_forecast_test.index = pd.to_datetime(mstep_forecast_test['ds'])
mstep_forecast_test = mstep_forecast_test.resample('MS').first()
mstep_forecast_test = np.exp(mstep_forecast_test[['yhat_lower', 'yhat_upper', 'yhat']]).rename(columns=rename_fbprophet)

mstep_forecast_test.tail()

In [ ]:
fig = plt.figure(figsize = (16,8))
ax = fig.add_subplot(111)

mstep_forecast_test.plot(y='p_cons_GNA95', label='forecast', ax=ax)
ax.fill_between(x=mstep_forecast_test.index, y1=mstep_forecast_test['upper p_cons_GNA95'], y2=mstep_forecast_test['lower p_cons_GNA95'],  color='k', alpha=0.1)
endog_GNA95_test['cons_GNA95'].plot(ax=ax, y='cons_GNA95', style='o', label='actual')
plt.legend()
plt.show()

In [ ]:
mle_tsa.compute_ape(endog_GNA95_test['cons_GNA95'], mstep_forecast_test['p_cons_GNA95']).mean()

# Single Step Forecasting

In [ ]:
forecast_mods, forecast_test = mle_tsa.prophet_endog_forecast(train_endog=endog_GNA95_train, test_endog=endog_GNA95_test)

In [ ]:
p_gna95_test = np.exp(forecast_test[['yhat_lower', 'yhat_upper', 'yhat']]).rename(columns=rename_fbprophet)

p_gna95_test.index = forecast_test['ds'].dt.date
p_gna95_test.index.name = 'Date'

p_gna95_test['cons_GNA95'] = endog_GNA95_test['cons_GNA95'].copy()

p_gna95_test['model'] = 'fbprophet'
p_gna95_test['split'] = 'test'

In [ ]:
fig = plt.figure(figsize = (16,8))
ax = fig.add_subplot(111)

p_gna95_test.plot(y='p_cons_GNA95', label='forecast', ax=ax)
ax.fill_between(x=p_gna95_test.index, y1=p_gna95_test['upper p_cons_GNA95'], y2=p_gna95_test['lower p_cons_GNA95'],  color='k', alpha=0.1)
p_gna95_test.plot(ax=ax, y='cons_GNA95', style='o', label='actual')
plt.suptitle("Forecasting Consumo GNA95")
plt.ylabel('t')
plt.legend()
plt.show()

# Model Assessment

In [ ]:
skl_mse(np.log(endog_GNA95_test['cons_GNA95']/1e3), forecast_test['yhat'])**0.5

In [ ]:
mle_tsa.compute_ape(p_gna95_test['cons_GNA95'], p_gna95_test['p_cons_GNA95']).mean()

In [ ]:
select_cols = ['model', 'split', 'cons_GNA95', 'p_cons_GNA95', 'lower p_cons_GNA95', 'upper p_cons_GNA95']

pd.concat([p_gna95_train, p_gna95_test], axis=0)[select_cols].to_excel(OUTPATH + OUTFILE, index_label='Date')